In [1]:
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
from skimage.measure import regionprops_table
from utils.nucStitch import fullNucImage16

In [175]:
indRainbowKI67Path= 'segmentations/inducedRainbowKI67/final/1-21'
indRainbowMappingPath = 'segmentations/inducedRainbow/final/red/0-5'
nucPath = 'segmentations/inducedRainbow/final/nuclei/0-5'

In [176]:
folders = [x[0] for x in os.walk(nucPath)]
folders = sorted(folders)

In [177]:
def clusterSize(row, dic):
    values = dic.get(row['label'],None)
    if values != None:
        return len(values)
    else:
        return 0

In [ ]:
dfMapping = pd.DataFrame()
for folder in folders:
    os.chdir(folder)
    files = [f for f in listdir(folder) if isfile(join(folder, f)) if ".DS_Store" not in f if ".npy" in f if "._" not in f]
    if files:
        nucleiPath = folder
        aorta = folder.split("/")[-1]
        age = folder.split("/")[-2]
        print(aorta)
        df = pd.DataFrame()
        nuclei = np.load(join(nucleiPath, "Cropped_IMG-1_seg.npy"), allow_pickle=True).item()['masks']
        yLength = nuclei.shape[0]
        xLength = nuclei.shape[1]
        for i in range(16):
            number = i+1
            nuclei = np.load(join(nucleiPath, "Cropped_IMG-" + str(number) + '_seg.npy'), allow_pickle=True).item()['masks']
            row = int(i/4) + 1
            column  = (i%4) + 1
            prop_dict = regionprops_table(nuclei, properties = ('label', 'centroid'))
            df_help = pd.DataFrame(prop_dict)
            df_help['picture_id'] = number
            df_help['yCoord'] = df_help['centroid-0']*row
            df_help['xCoord'] = df_help['centroid-1']*column
            #df = df.append(df_help, ignore_index=True)
            df = pd.concat([df, df_help], axis=0)

        nucImage = fullNucImage16(nucleiPath)
        segFile = files = [f for f in listdir(join(indRainbowMappingPath, aorta)) if isfile(join(indRainbowMappingPath, aorta, f)) if ".DS_Store" not in f if ".npy" in f if "._" not in f if "cellposeSeg" in f]
        if segFile:
            masks = np.load(join(indRainbowMappingPath, aorta,segFile[0]), allow_pickle=True).item()['masks']
            accuracy_value = 0.8
            clusters = list()
            dic = {}
            dicNot = list()
            for mask in np.unique(masks):
                if mask != 0:
                    cluster = list()
                    coord = np.where(masks == mask)
                    segmentationNuc = np.unique(nucImage[coord])
                    for j in segmentationNuc:
                        if j != 0:
                            #occurence of nucleus pixel in
                            nucOccur = np.bincount(nucImage[coord])[j]
                            #How many pixels does this nucleus have:
                            coordLen = len(np.where(nucImage == j)[0])
                            #check if most of nuclei values are in segmentation
                            if(nucOccur >= accuracy_value*coordLen):
                                cluster.append(j)
                    if cluster:
                        dic[mask] = cluster
                        clusters.append(cluster)
                    else:
                        dicNot.append(mask)
            prop_dict = regionprops_table(masks, properties = ('label', 'area'))
            dfArea = pd.DataFrame(prop_dict)
            dfArea['clusterSize'] = dfArea.apply(lambda row: clusterSize(row, dic), axis=1)
            dfArea['age'] = age
            dfArea['aorta'] = aorta
            dfMapping = pd.concat([dfMapping, dfArea])



In [180]:
dfMappingSave = dfMapping.loc[dfMapping.clusterSize != 0]

In [182]:
dfMappingSave.to_csv("/Volumes/Aorta/aorta/results/inducedRainbowKI67/mappingP0-5.csv", index=False)

Mapp Ki67Active areas

In [83]:
conversion = 1.7199802336410723

In [84]:
conversion2 = 0.8599901168205362

In [87]:
indRainbowKI67Path= 'results/inducedRainbowKI67/resultsArea.csv'

In [2]:
dfMapping = pd.read_csv("/results/inducedRainbowKI67/mappingP0-5.csv")

In [3]:
np.unique(dfMapping['clusterSize'])

array([ 1,  2,  3,  4,  5,  6,  7,  8, 12])

In [7]:
np.min(dfMapping['area'])

15

In [4]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 1]['area'], 0.75)

131.0

In [5]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 2]['area'], 0.25)

152.0

In [12]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 2]['area'], 0.7)

239.59999999999997

In [13]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 3]['area'], 0.3)

247.4

In [20]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 3]['area'], 0.6)

334.6

In [21]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 4]['area'], 0.4)

326.4

In [26]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 4]['area'], 0.7)

406.4

In [27]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 5]['area'], 0.3)

421.0

In [30]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 5]['area'], 0.7)

513.3

In [31]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 6]['area'], 0.3)

518.2

In [32]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 6]['area'], 0.75)

664.0

In [33]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 7]['area'], 0.25)

661.0

In [40]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 7]['area'], 0.75)

740.5

In [41]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 8]['area'], 0.75)

728.5

In [139]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 8]['area'], 0.5)

1366.0

In [143]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 9]['area'], 0.5)

1176.0

In [148]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 9]['area'], 0.75)

1612.0

In [149]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 10]['area'], 0.3)

1641.0

In [155]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 10]['area'], 0.9)

1894.0

In [154]:
np.quantile(dfMapping.loc[dfMapping.clusterSize == 11]['area'], 0.9)

1791.0

In [156]:
np.min(dfMapping['area'])

19

In [157]:
np.max(dfMapping['area'])

2190

In [158]:
dfMapping

,label,area,clusterSize,age,aorta
0,13,95,1,5-60,aorta04
1,15,289,2,5-60,aorta04
2,16,232,2,5-60,aorta04
3,17,289,2,5-60,aorta04
4,30,44,1,5-60,aorta04
...,...,...,...,...,...
32555,1310,309,1,5-60,aorta08
32556,1311,837,3,5-60,aorta08
32557,1315,863,4,5-60,aorta08
32558,1318,332,1,5-60,aorta08
